In [1]:
import numpy as np
import pandas as pd
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/home/cseadmin/dz/TrafficFlowModel/model'

In [2]:
embed_matrix=np.load("./saved/embedding_matrix.npy")

embed_matrix.shape

(492, 64)

In [3]:
cor_matrix=embed_matrix @ embed_matrix.T

cor_matrix.shape

(492, 492)

In [4]:
cor_matrix

array([[72.30105  , -0.386034 , 12.120458 , ..., -5.0983267, 10.383698 ,
         0.       ],
       [-0.386034 , 73.07714  , -8.034481 , ..., -3.3336499,  6.7834587,
         0.       ],
       [12.120458 , -8.034481 , 70.43657  , ..., -6.648742 ,  6.468872 ,
         0.       ],
       ...,
       [-5.0983267, -3.3336499, -6.648742 , ..., 76.912605 ,  2.3218575,
         0.       ],
       [10.383698 ,  6.7834587,  6.468872 , ...,  2.3218575, 77.01283  ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [15]:
# sum_of_rows=cor_matrix.sum(axis=1)
# cor_matrix/=sum_of_rows[:, np.newaxis]
# np.nan_to_num(cor_matrix, copy=False)

/home/cseadmin/dz/anaconda3/envs/torch1.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([[ 0.18448809,  0.00974219,  0.01123869, ...,  0.00293595,
         0.00094138,  0.        ],
       [ 0.01159629,  0.20665924,  0.04856317, ...,  0.02418181,
         0.01470489,  0.        ],
       [ 0.00733121,  0.02661366,  0.10077152, ...,  0.010441  ,
        -0.00537754,  0.        ],
       ...,
       [ 0.00251128,  0.01737695,  0.01369081, ...,  0.13979769,
         0.00495891,  0.        ],
       [ 0.00074642,  0.00979532, -0.00653646, ...,  0.00459683,
         0.12951389,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [5]:
np.save("../../DL-Traff-Graph/SZTAXI/cor_matrix.npy", cor_matrix, allow_pickle=True)

---

In [52]:
# k=4
# idx = np.argsort(cor_matrix, axis=1)[:,-k:]
# idx

array([[398, 106, 105,   0],
       [ 84, 401,  29,   1],
       [144, 127, 192,   2],
       ...,
       [208, 224, 169, 489],
       [425, 468, 243, 490],
       [156, 155, 153, 491]])

In [53]:
# knn_adj=np.zeros((492, 492))
# for i in range(len(idx)):
#     knn_adj[i][idx[i]]=1
    
# knn_adj

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [54]:
# np.save(f"../../DL-Traff-Graph/SZTAXI/adj_{k}.npy", knn_adj, allow_pickle=True)

In [ ]:
for k in range(1, 21):
    idx = np.argsort(cor_matrix, axis=1)[:,-k:]
    knn_adj=np.zeros((492, 492))
    for i in range(len(idx)):
        knn_adj[i][idx[i]]=1
    np.save(f"../../DL-Traff-Graph/SZTAXI/adj_{k}.npy", knn_adj, allow_pickle=True)

---

In [29]:
rel=pd.read_csv("../data/sz_taxi_202006/sz_taxi_202006.rel")

rel

,rel_id,type,origin_id,destination_id
0,0,geo,0,70
1,1,geo,0,246
2,2,geo,0,252
3,3,geo,1,2
4,4,geo,1,63
...,...,...,...,...
1689,1689,geo,490,244
1690,1690,geo,490,488
1691,1691,geo,491,245
1692,1692,geo,491,489


In [30]:
adj=np.zeros((492, 492))
adj_sym=np.zeros((492, 492))

for index, row in rel.iterrows():
    u=row["origin_id"]
    v=row["destination_id"]
    adj[u][v]=1
    
    adj_sym[u][v]=1
    adj_sym[v][u]=1
    
adj
adj_sym

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.]])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [35]:
np.sum(adj[0]==1)
adj[0][105]

3

0.0

In [8]:
np.save("../data/sz_taxi_202006/adj_mx.npy", adj)
np.save("../data/sz_taxi_202006/adj_sym.npy", adj_sym)

---

In [2]:
def softmax(x):
    """ softmax function """
    
    # assert(len(x.shape) > 1, "dimension must be larger than 1")
    # print(np.max(x, axis = 1, keepdims = True)) # axis = 1, 行
    
    x1 = x
    x1 -= np.max(x, axis = 1, keepdims = True) #为了稳定地计算softmax概率， 一般会减掉最大的那个元素
    x1 = np.exp(x1) / np.sum(np.exp(x1), axis = 1, keepdims = True)
    
    return x1

cor_matrix=np.load("../../DL-Traff-Graph/SZTAXI/cor_matrix.npy", allow_pickle=True)
OD_matrix=np.load("../../DL-Traff-Graph/SZTAXI/OD_matrix.npy", allow_pickle=True)

In [5]:
OD_matrix=OD_matrix.astype(np.float32)
np.save("../../DL-Traff-Graph/SZTAXI/OD_matrix.npy", OD_matrix)

In [21]:
for i in range(len(cor_matrix)):
    cor_matrix[i, i]=0

cor_matrix_sm=softmax(cor_matrix)

for i in range(len(cor_matrix_sm)):
    cor_matrix_sm[i, i]=1

cor_matrix_sm

array([[1.0000000e+00, 9.9820000e-14, 2.6959899e-08, ..., 8.9679931e-16,
        4.7473732e-09, 1.4684865e-13],
       [7.3267905e-13, 1.0000000e+00, 3.4932965e-16, ..., 3.8439717e-14,
        9.5188590e-10, 1.0778697e-12],
       [2.5284483e-09, 4.4634860e-18, 1.0000000e+00, ..., 1.7844089e-17,
        8.8797606e-12, 1.3772278e-14],
       ...,
       [7.9176808e-17, 4.6236686e-16, 1.6798144e-17, ..., 1.0000000e+00,
        1.3217296e-13, 1.2965006e-14],
       [2.5933127e-09, 7.0841909e-11, 5.1721006e-11, ..., 8.1778925e-13,
        1.0000000e+00, 8.0217944e-14],
       [2.0325202e-03, 2.0325202e-03, 2.0325202e-03, ..., 2.0325202e-03,
        2.0325202e-03, 1.0000000e+00]], dtype=float32)

In [4]:
OD_matrix

array([[1.660e+02, 0.000e+00, 1.000e+00, ..., 2.700e+01, 5.570e+02,
        8.400e+01],
       [1.600e+01, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.400e+01,
        1.000e+00],
       [8.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.800e+01, 0.000e+00, 0.000e+00, ..., 2.400e+01, 6.110e+02,
        1.010e+02],
       [5.100e+01, 0.000e+00, 0.000e+00, ..., 1.321e+03, 7.630e+02,
        8.600e+01],
       [2.000e+00, 0.000e+00, 0.000e+00, ..., 8.000e+00, 6.810e+02,
        1.000e+00]], dtype=float32)

In [23]:
OD_matrix_sm=softmax(OD_matrix)

OD_matrix_sm

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.8003888e-35, 2.0248763e-42, 2.0248763e-42, ..., 2.0248763e-42,
        2.4365609e-36, 5.5071030e-42],
       [8.3151400e-07, 2.7894190e-10, 2.7894190e-10, ..., 2.7894190e-10,
        2.7894190e-10, 2.7894190e-10],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [26]:
np.save("../../DL-Traff-Graph/SZTAXI/cor_matrix_sm.npy", cor_matrix_sm)
np.save("../../DL-Traff-Graph/SZTAXI/OD_matrix_sm.npy", OD_matrix_sm)

Feature Person

---

In [2]:
flow_mat=pd.read_pickle("../../DL-Traff-Graph/SZTAXI/SZTAXI-flow.pkl").values.astype(np.float32)
speed_mat=pd.read_pickle("../../DL-Traff-Graph/SZTAXI/SZTAXI-speed.pkl").values.astype(np.float32)

flow_mat.shape
speed_mat.shape

(8064, 492)

(8064, 492)

In [7]:
flow_mat

array([[21.,  0.,  0., ...,  0., 11.,  6.],
       [18.,  0.,  0., ...,  5., 12.,  8.],
       [27.,  0.,  0., ..., 10., 11.,  8.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [8]:
speed_mat

array([[41.235294, 42.      , 34.      , ..., 38.      , 38.909092,
        45.166668],
       [29.916666, 29.339838, 29.339838, ..., 36.5     , 38.083332,
        42.      ],
       [31.142857, 20.5     , 29.339838, ..., 31.666666, 41.454544,
        41.25    ],
       ...,
       [29.339838, 29.339838, 29.339838, ..., 29.339838, 29.339838,
        29.339838],
       [29.339838, 29.339838, 29.339838, ..., 29.339838, 29.339838,
        29.339838],
       [29.339838, 29.339838, 29.339838, ..., 29.339838, 29.339838,
        29.339838]], dtype=float32)

In [3]:
flow_pearson=np.corrcoef(flow_mat.T, rowvar=True)
speed_pearson=np.corrcoef(speed_mat.T, rowvar=True)

flow_pearson=np.nan_to_num(flow_pearson)
speed_pearson=np.nan_to_num(speed_pearson)

flow_pearson.shape

/home/cseadmin/dz/anaconda3/envs/torch1.7/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/cseadmin/dz/anaconda3/envs/torch1.7/lib/python3.7/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


(492, 492)

In [4]:
np.save("../../DL-Traff-Graph/SZTAXI/flow_pearson.npy", flow_pearson)
np.save("../../DL-Traff-Graph/SZTAXI/speed_pearson.npy", speed_pearson)

Feature Cov

---

In [5]:
flow_cov=np.cov(flow_mat.T, rowvar=True)
speed_cov=np.cov(speed_mat.T, rowvar=True)

flow_cov.shape
flow_cov

(492, 492)

array([[68.88634627,  1.86080194,  2.82013712, ...,  5.03896222,
        32.47346372, 33.14786318],
       [ 1.86080194,  1.74884091,  0.26197988, ...,  0.24180213,
         2.10523102,  2.13515167],
       [ 2.82013712,  0.26197988,  2.4011883 , ...,  0.19914075,
         2.16755162,  2.23961886],
       ...,
       [ 5.03896222,  0.24180213,  0.19914075, ..., 12.52190044,
         4.77664591,  4.5409753 ],
       [32.47346372,  2.10523102,  2.16755162, ...,  4.77664591,
        74.64525733, 67.44690999],
       [33.14786318,  2.13515167,  2.23961886, ...,  4.5409753 ,
        67.44690999, 70.82041939]])

In [6]:
np.save("../../DL-Traff-Graph/SZTAXI/flow_cov.npy", flow_cov)
np.save("../../DL-Traff-Graph/SZTAXI/speed_cov.npy", speed_cov)

Feature Cosine

---

In [9]:
def get_cos_similar_matrix(v1, v2):
    num = np.dot(v1, np.array(v2).T)  # 向量点乘
    denom = np.linalg.norm(v1, axis=1).reshape(-1, 1) * np.linalg.norm(v2, axis=1)  # 求模长的乘积
    res = num / denom
    res[np.isneginf(res)] = 0
    return 0.5 + 0.5 * res

In [10]:
flow_cosine=get_cos_similar_matrix(flow_mat.T, flow_mat.T)
speed_cosine=get_cos_similar_matrix(speed_mat.T, speed_mat.T)

flow_cosine=np.nan_to_num(flow_cosine)
speed_cosine=np.nan_to_num(speed_cosine)

flow_cosine.shape
flow_cosine
speed_cosine

/home/cseadmin/dz/anaconda3/envs/torch1.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


(492, 492)

array([[1.        , 0.63321173, 0.6621315 , ..., 0.8349408 , 0.9434126 ,
        0.9400189 ],
       [0.63321173, 1.        , 0.58899283, ..., 0.5925163 , 0.63988054,
        0.64256835],
       [0.6621315 , 0.58899283, 1.        , ..., 0.6014187 , 0.6528029 ,
        0.655298  ],
       ...,
       [0.8349408 , 0.5925163 , 0.6014187 , ..., 1.        , 0.82083976,
        0.8158041 ],
       [0.9434126 , 0.63988054, 0.6528029 , ..., 0.82083976, 1.        ,
        0.9901964 ],
       [0.9400189 , 0.64256835, 0.655298  , ..., 0.8158041 , 0.9901964 ,
        1.        ]], dtype=float32)

array([[0.9999989 , 0.9784517 , 0.97766286, ..., 0.9891083 , 0.9898609 ,
        0.99092036],
       [0.9784517 , 0.9999896 , 0.966593  , ..., 0.9759396 , 0.97659755,
        0.9775419 ],
       [0.97766286, 0.966593  , 0.99999523, ..., 0.975599  , 0.97720474,
        0.97752976],
       ...,
       [0.9891083 , 0.9759396 , 0.975599  , ..., 1.0000013 , 0.9880032 ,
        0.98949635],
       [0.9898609 , 0.97659755, 0.97720474, ..., 0.9880032 , 1.0000007 ,
        0.9980149 ],
       [0.99092036, 0.9775419 , 0.97752976, ..., 0.98949635, 0.9980149 ,
        1.000001  ]], dtype=float32)

In [11]:
np.save("../../DL-Traff-Graph/SZTAXI/flow_cosine.npy", flow_cosine)
np.save("../../DL-Traff-Graph/SZTAXI/speed_cosine.npy", speed_cosine)

Feature DTW

---

In [13]:
from fastdtw import fastdtw

flow_DTW=np.zeros((492, 492))
speed_DTW=np.zeros((492, 492))

for i in range(492):
    for j in range(492):
        flow_DTW[i, j]=fastdtw(flow_mat[:, i], flow_mat[:, j])[0]
        speed_DTW[i, j]=fastdtw(speed_mat[:, i], speed_mat[:, j])[0]
        
speed_DTW

array([[     0.        ,  54497.34051108,  57320.01793623, ...,
         35878.12430239,  66807.69944668,  71687.07831573],
       [ 52276.80829597,      0.        ,  49653.66195893, ...,
         57726.71536088,  91890.21368194,  97642.92257857],
       [ 58072.0928874 ,  50012.60881257,      0.        , ...,
         66273.06735754,  95592.54577875, 113863.69368029],
       ...,
       [ 36309.2448926 ,  60729.9672873 ,  64875.01386309, ...,
             0.        ,  71183.55299807,  76568.64624834],
       [ 64173.25999737,  94739.09312892,  99816.87948847, ...,
         71686.3544383 ,      0.        ,  28235.60124302],
       [ 73386.24558544,  99918.05470157, 115379.45998907, ...,
         77134.22741175,  28025.10507584,      0.        ]])

In [19]:
for i in range(492):
    speed_DTW[i, i]=0

In [32]:
speed_DTW_new=speed_DTW
speed_DTW_new=np.round(speed_DTW_new)
speed_DTW_new/=10

speed_DTW_new

array([[    0. ,  5449.7,  5732. , ...,  3587.8,  6680.8,  7168.7],
       [ 5227.7,     0. ,  4965.4, ...,  5772.7,  9189. ,  9764.3],
       [ 5807.2,  5001.3,     0. , ...,  6627.3,  9559.3, 11386.4],
       ...,
       [ 3630.9,  6073. ,  6487.5, ...,     0. ,  7118.4,  7656.9],
       [ 6417.3,  9473.9,  9981.7, ...,  7168.6,     0. ,  2823.6],
       [ 7338.6,  9991.8, 11537.9, ...,  7713.4,  2802.5,     0. ]])

In [33]:
# np.save("../../DL-Traff-Graph/SZTAXI/flow_DTW.npy", flow_DTW)
np.save("../../DL-Traff-Graph/SZTAXI/speed_DTW.npy", speed_DTW_new)

---

In [4]:
print(np.load("../../DL-Traff-Graph/SZTAXI/flow_pearson.npy"))
print("---")
print(np.load("../../DL-Traff-Graph/SZTAXI/flow_cosine.npy"))

[[1.         0.16953463 0.21927572 ... 0.17156923 0.45285684 0.47458031]
 [0.16953463 1.         0.1278438  ... 0.05167135 0.18425679 0.19185553]
 [0.21927572 0.1278438  1.         ... 0.03631716 0.16190312 0.17174407]
 ...
 [0.17156923 0.05167135 0.03631716 ... 1.         0.15623805 0.15248768]
 [0.45285684 0.18425679 0.16190312 ... 0.15623805 1.         0.92764477]
 [0.47458031 0.19185553 0.17174407 ... 0.15248768 0.92764477 1.        ]]
---
[[1.         0.63321173 0.6621315  ... 0.8349408  0.9434126  0.9400189 ]
 [0.63321173 1.         0.58899283 ... 0.5925163  0.63988054 0.64256835]
 [0.6621315  0.58899283 1.         ... 0.6014187  0.6528029  0.655298  ]
 ...
 [0.8349408  0.5925163  0.6014187  ... 1.         0.82083976 0.8158041 ]
 [0.9434126  0.63988054 0.6528029  ... 0.82083976 1.         0.9901964 ]
 [0.9400189  0.64256835 0.655298   ... 0.8158041  0.9901964  1.        ]]
